In [163]:
import inspect
from berny import Berny, geomlib, coords,Geometry,Math
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [164]:
def mpa(g,h,R0,De): # Morse Potential Approximation
    Z=g**2/(2*De)/h
    for i in range(3):
        t=float(np.roots([1,-2,1-2*Z,+Z])[i])
        print(np.roots([1,-2,1-2*Z,+Z]))
        a=g/(2*De)/(t*(1-t))
        print(a)
        dX=np.log(t)/a
        if a>0 and t>0:
            return R0+dX
    return -1

In [165]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [166]:
h2o=gto.M(atom="O 0 0 0; H 0 -0.7 .3; H 0 0.7 .3",basis="def2-TZVP")

In [167]:
mf=scf.RHF(h2o)
E0=mf.scf()
g0=mf.Gradients().grad()

converged SCF energy = -75.9417083056576
--------------- RHF gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0000000000     0.3591502074
1 H    -0.0000000000     0.3476755730    -0.1795751037
2 H     0.0000000000    -0.3476755730    -0.1795751037
----------------------------------------------


In [168]:
geom2=Geometry([h2o.atom_symbol(x) for x in range(h2o.natm)],h2o.atom_coords()*bohr2ang)

In [169]:
anglecenter(h2o.atom_coords()) ,-0.0296174*180/np.pi

(133.603, -1.6969520201507642)

In [170]:
b2=Berny(geom2)

In [171]:
s=b2._state
vars(b2._state)

{'geom': <Geometry 'H2O'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 2, angles: 1, dihedrals: 0">,
 'H': array([[0.61593528, 0.        , 0.        ],
        [0.        , 0.61593528, 0.        ],
        [0.        , 0.        , 0.28101946]]),
 'weights': array([1.36874507, 1.36874507, 1.03647041]),
 'future': Point(q=array([1.43917254, 1.43917254, 2.33180908]), E=None, g=None),
 'first': True}

In [172]:
print(dir(b2._state.geom))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'bondmatrix', 'cms', 'coords', 'copy', 'dist', 'dist_diff', 'dump', 'dumps', 'formula', 'from_atoms', 'inertia', 'lattice', 'masses', 'rho', 'species', 'super_circum', 'supercell', 'write']


In [173]:
vars(b2._state.coords)

{'_coords': [Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)],
 'fragments': [[0, 1, 2]]}

In [174]:
b2._state.coords._coords

[Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)]

In [175]:
vars(b2._state.coords._coords[0])

{'i': 0, 'j': 1, 'idx': (0, 1), 'weak': 0}

In [176]:
b2._state.H  #use to get second derivs 

array([[0.61593528, 0.        , 0.        ],
       [0.        , 0.61593528, 0.        ],
       [0.        , 0.        , 0.28101946]])

In [177]:
vars(geom2)

{'species': ['O', 'H', 'H'], 'coords': array([[ 0. ,  0. ,  0. ],
        [ 0. , -0.7,  0.3],
        [ 0. ,  0.7,  0.3]]), 'lattice': None}

In [178]:
b2._state.future.q

array([1.43917254, 1.43917254, 2.33180908])

In [179]:
np.arccos(-.40/0.58),(.58)**.5*ang2bohr

(2.3318090810196264, 1.4391725397177106)

In [180]:
# get derivatives angles independently
# 

In [181]:
b2._state.future

Point(q=array([1.43917254, 1.43917254, 2.33180908]), E=None, g=None)

In [182]:
print(s.coords.B_matrix(s.geom))

[[ 0.          0.91914503 -0.3939193   0.         -0.91914503  0.3939193
   0.          0.          0.        ]
 [ 0.         -0.91914503 -0.3939193   0.          0.          0.
   0.          0.91914503  0.3939193 ]
 [ 0.          0.          1.2773243   0.         -0.27371235 -0.63866215
   0.          0.27371235 -0.63866215]]


In [183]:
gsc=b2._state.geom.supercell()

In [184]:
vars(gsc),vars(s.geom)

({'species': ['O', 'H', 'H'], 'coords': array([[ 0. ,  0. ,  0. ],
         [ 0. , -0.7,  0.3],
         [ 0. ,  0.7,  0.3]]), 'lattice': None},
 {'species': ['O', 'H', 'H'], 'coords': array([[ 0. ,  0. ,  0. ],
         [ 0. , -0.7,  0.3],
         [ 0. ,  0.7,  0.3]]), 'lattice': None})

In [185]:
[print(x) for x in enumerate(s.coords)]

(0, Bond(0, 1, weak=0))
(1, Bond(0, 2, weak=0))
(2, Angle(1, 0, 2, weak=0))


[None, None, None]

In [186]:
B=s.coords.B_matrix(gsc)

In [187]:
for i,x in enumerate(s.coords):
    print(x.eval(gsc.coords, grad=True)[1],"\n \n")

[array([ 0.        ,  0.91914503, -0.3939193 ]), array([-0.        , -0.91914503,  0.3939193 ])] 
 

[array([ 0.        , -0.91914503, -0.3939193 ]), array([-0.        ,  0.91914503,  0.3939193 ])] 
 

[array([-0.        , -0.27371235, -0.63866215]), array([0.       , 0.       , 1.2773243]), array([-0.        ,  0.27371235, -0.63866215])] 
 



### praticamente qua mi d una proiezione dei modi normali sulle coordinate cartesiane della molecola, tutti insieme formano la B_matrix

In [188]:
a=np.array([-0.        , -0.91914503,  0.3939193 ])

In [189]:
norm(a)

1.0000000005430953

In [190]:
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))

In [191]:
B_inv,B_inv.shape,B.shape

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.41733409e-01, -3.41733409e-01, -1.11022302e-16],
        [-1.31306433e-01, -1.31306433e-01,  4.40936096e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.30439219e-01, -2.88705811e-01, -2.83458919e-01],
        [ 1.38881804e-01, -7.57537113e-03, -2.20468048e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.88705811e-01,  6.30439219e-01,  2.83458919e-01],
        [-7.57537113e-03,  1.38881804e-01, -2.20468048e-01]]), (9, 3), (3, 9))

In [192]:
np.dot(B_inv.T, g0.reshape(-1))

array([-0.39030237, -0.39030237,  0.04043995])

In [193]:
s.H

array([[0.61593528, 0.        , 0.        ],
       [0.        , 0.61593528, 0.        ],
       [0.        , 0.        , 0.28101946]])

I can have the projection of the gradient in the internal coordinates as well as the approximated hessian matrix.

In [194]:
proj = np.dot(B, B_inv)
proj,B@B_inv

(array([[1.00000000e+00, 3.88578059e-16, 1.80411242e-16],
        [1.97758476e-16, 1.00000000e+00, 4.85722573e-16],
        [8.93382590e-17, 2.63677968e-16, 1.00000000e+00]]),
 array([[1.00000000e+00, 3.88578059e-16, 1.80411242e-16],
        [1.97758476e-16, 1.00000000e+00, 4.85722573e-16],
        [8.93382590e-17, 2.63677968e-16, 1.00000000e+00]]))

In [195]:
H_proj = proj.dot(s.H).dot(proj) + 1000 * (np.eye(len(s.coords)) - proj)
1000 * (np.eye(len(s.coords)) - proj)  #perhaps to avoid zeros in matrix inversion

array([[ 0.00000000e+00, -3.88578059e-13, -1.80411242e-13],
       [-1.97758476e-13, -2.22044605e-13, -4.85722573e-13],
       [-8.93382590e-14, -2.63677968e-13,  0.00000000e+00]])

In [196]:
H_proj,s.H #differiscono per una manciata di infinesimi

(array([[ 6.15935283e-01, -3.88099381e-13, -1.80249421e-13],
        [-1.97514863e-13,  6.15935283e-01, -4.85286902e-13],
        [-8.92581266e-14, -2.63441461e-13,  2.81019461e-01]]),
 array([[0.61593528, 0.        , 0.        ],
        [0.        , 0.61593528, 0.        ],
        [0.        , 0.        , 0.28101946]]))

In [197]:
def quadratic_step(g, H, w, trust):
    ev = np.linalg.eigvalsh((H + H.T) / 2)
    rfo = np.vstack((np.hstack((H, g[:, None])), np.hstack((g, 0))[None, :]))
    D, V = np.linalg.eigh((rfo + rfo.T) / 2)
    dq = V[:-1, 0] / V[-1, 0]
    l = D[0]
    if norm(dq) <= trust:
        print('Pure RFO step was performed:')
        on_sphere = False
    else:
        def steplength(l):
            return norm(np.linalg.solve(l * np.eye(H.shape[0]) - H, g)) - trust
        l = Math.findroot(steplength, ev[0])  # minimization on sphere
        dq = np.linalg.solve(l * np.eye(H.shape[0]) - H, g)
        on_sphere = True
        print('Minimization on sphere was performed:')
    dE = np.dot(g, dq) + 0.5 * dq.dot(H).dot(dq)
    return dq, dE, on_sphere

In [198]:
current = Berny.Point(s.future.q, mf.e_tot, np.dot(B_inv.T, np.asarray(g0*bohr2ang).reshape(-1)))

In [199]:
current.g # also np.dot(B_inv.T, np.asarray(g0).reshape(-1)) is g in internal coords

array([-0.20653912, -0.20653912,  0.0213999 ])

In [200]:
 dq, dE, on_sphere = quadratic_step(np.dot(proj, current.g), H_proj, s.weights, s.trust)

Minimization on sphere was performed:


In [201]:
dq # change dq to the one from Morse potential fitting 

array([ 0.21082967,  0.21082967, -0.03319186])

In [204]:
current.q[0]

1.4391725396367097

In [226]:
mpa(current.g[0]*bohr2ang,s.H[0,0],current.q[0],.1)*bohr2ang

[ 1.34939895  0.74682489 -0.09622384]
1.1590749037528087


0.8983871543287781

## from dq in internal coords to dq in cartesian coords

In [214]:
q, a = s.coords.update_geom(s.geom, current.q, dq, B_inv)

In [215]:
q

array([1.65000222, 1.65000222, 2.29861736])

In [216]:
a.coords

array([[ 0.00000000e+00,  5.19744548e-16, -3.81460429e-02],
       [ 0.00000000e+00, -7.96727207e-01,  3.19073021e-01],
       [ 0.00000000e+00,  7.96727207e-01,  3.19073021e-01]])

In [217]:
b=a.coords
norm(b[1]-b[0]) #is angstrom

0.8731435747133777

In [218]:
h2o=gto.M(atom="O {} {} {};H {} {} {}; H {} {} {}".format(*b.reshape(-1).tolist()),basis="def2-TZVP")

In [219]:
mf=scf.RHF(h2o)
E0=mf.scf()

converged SCF energy = -76.0365645692517


In [220]:
dE , 75.9417083056576 -76.0363277396315

(-0.06026685324900208, -0.09461943397390371)

In [221]:
eq_h2o=optimize(mf)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000  -0.038146    0.000000  0.000000  0.000000
   H   0.000000  -0.796727   0.319073    0.000000  0.000000  0.000000
   H   0.000000   0.796727   0.319073    0.000000  0.000000  0.000000
converged SCF energy = -76.0365645692529
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000     0.0000000000     0.1276700592
1 H    -0.0000000000     0.0695784153    -0.0638350296
2 H    -0.0000000000    -0.0695784153    -0.0638350296
----------------------------------------------
cycle 1: E = -76.0365645693  dE = -76.0366  norm(grad) = 0.184748

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000  -0.000000  -0.112319    0.000000 -0.000000 -0.074173
   H   0.000000  -0.829993   0.356160    0.000

In [222]:
norm(eq_h2o.atom_coords()[1]-eq_h2o.atom_coords()[0])*bohr2ang

0.9420488553881738

In [223]:
ecoords=eq_h2o.atom_coords()-eq_h2o.atom_coords()[0]

In [224]:
anglecenter(ecoords)

106.507